这是一个二分类文本示例，来自tensorflow的官方教程，用于入门自然语言处理。使用IMDB数据集，训练集25000条，测试集25000条。包含正面影评和负面影评。

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.13.1


In [3]:
# 下载IMDB数据集

imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 10s 1us/step


In [6]:
# 观察数据
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[0])
print(len(train_data[0]), len(train_data[1]))

Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
218 189


观看上面的输出可以看到，文本数据已经被处理成了整数，每个整数代表的是一个词，其实应该是该词在词典中的序号。

In [17]:
# 读取词典，可以将整数转换为文本
word_index = imdb.get_word_index()

# 字典中预留了一部分非文字类的词，比如标识符<start>, 还有UNK等
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

# 该字典用于将index转换为文字
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index[i] for i in text])

In [18]:
print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

# 准备数据
影评（整数数组）必须转换为张量，且必须具有相同的长度，然后才能馈送到神经网络中。使用onehot编码会导致占用大量闪存，我们可以使用填充数组的方法。

In [19]:
# 使用<PAD>补齐不够长的数组，设置maxlen为256，则所有向量的长度均为256
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [22]:
print(len(train_data[0]), len(train_data[1]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

# 构建模型
输入数据已经由文本转换为对应的索引数组，预测的标签为0或1

第一层是Embedding层，该层会在整数编码的词汇表中查找每个字词-索引的嵌入向量。模型在接受训练时会学习这些向量。这些向量会向输出数组添加一个维度。生成的维度为: (batch, sequence, embedding)

GlobalAveragePooling1D层对序列维度求平均值，也就是将每句话的所有词向量进行加和求平均，每个训练语句就可以被表示成长度相同的一维向量。

然后输入到一个全连接层，激活函数为relu。
最后一层输出结果，使用sigmoid激活函数，计算置信水平。

In [24]:
# 设置最长词典大小
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


# 设置损失函数和优化器

损失函数使用交叉熵crossentropy， 评价函数为acc， 即预测准确的样本占总样本数量。

In [26]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 训练

In [27]:
# 创建验证集
#取10000个数据作为验证集

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [28]:
# 训练
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
15000/15000 [==============================] - 2s 149us/sample - loss: 0.6918 - acc: 0.6125 - val_loss: 0.6894 - val_acc: 0.6356
Epoch 2/40
15000/15000 [==============================] - 1s 67us/sample - loss: 0.6845 - acc: 0.7227 - val_loss: 0.6791 - val_acc: 0.7523
Epoch 3/40
15000/15000 [==============================] - 1s 68us/sample - loss: 0.6685 - acc: 0.7674 - val_loss: 0.6593 - val_acc: 0.7637
Epoch 4/40
15000/15000 [==============================] - 1s 66us/sample - loss: 0.6415 - acc: 0.7763 - val_loss: 0.6290 - val_acc: 0.7674
Epoch 5/40
15000/15000 [==============================] - 1s 66us/sample - loss: 0.6033 - acc: 0.8015 - val_loss: 0.5901 - val_acc: 0.7934
Epoch 6/40
15000/15000 [==============================] - 1s 67us/sample - loss: 0.5572 - acc: 0.8213 - val_loss: 0.5468 - val_acc: 0.8109
Epoch 7/40
15000/15000 [==============================] - 1s 67us/s

# 预测

In [29]:
results = model.evaluate(test_data, test_labels)
print(results)

25000/25000 [==============================] - 0s 15us/sample - loss: 0.3347 - acc: 0.8710
[0.3347101798057556, 0.87104]
